In [2]:
import numpy as np
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [3]:
from graph_attention_layer import GraphAttention
from  untils import load_data,preprocess_features

In [4]:
# Read data
A, X, Y_train, Y_val, Y_test, idx_train, idx_val, idx_test = load_data('cora')

In [5]:
print('X的shape:',X.shape)
print('A的shape:',A.shape)
print('Y_train的shape:',Y_train.shape)

X的shape: (2708, 1433)
A的shape: (2708, 2708)
Y_train的shape: (2708, 7)


In [12]:
# Parameters
N = X.shape[0]                # Number of nodes in the graph
F = X.shape[1]                # Original feature dimension
n_classes = Y_train.shape[1]  #标签的维度
F_ = 8                        # Output size of first GraphAttention layer
n_attn_heads = 8              # Number of attention heads  即K值
dropout_rate = 0.6            # Dropout rate (between and inside GAT layers)
l2_reg = 5e-4/2
learning_rate = 5e-3
epochs = 10
es_patience = 100


In [7]:
# 对行进行规范化
X= preprocess_features(X)

In [8]:
X.shape

(2708, 1433)

In [9]:
A = A +np.eye(A.shape[0])  #加上自连接

In [10]:
## 定义模型输入
X_in = Input(shape=(F,))
A_in = Input(shape=(N,))

dropout1 = Dropout(dropout_rate)(X_in)

#第一次注意力层
graph_attention_1 = GraphAttention(F_,
                                  attn_heads=n_attn_heads,
                                  dropout_rate=dropout_rate,
                                  activation='elu',
                                  kernel_regularizer=l2(l2_reg),
                                  attn_kernel_regularizer=l2(l2_reg))([dropout1, A_in])


dropout2 = Dropout(dropout_rate)(graph_attention_1)

#第二次注意力层
graph_attention_2 = GraphAttention(n_classes,
                                  attn_heads=1,
                                  attn_heads_reduction='average',
                                  dropout_rate=dropout_rate,
                                  activation='softmax',
                                  kernel_regularizer=l2(l2_reg),
                                  attn_kernel_regularizer=l2(l2_reg))([dropout2, A_in])

model = Model(inputs=[X_in, A_in], outputs=graph_attention_2)

optimizer = Adam(lr=learning_rate)
model.compile(optimizer = optimizer,
             loss='categorical_crossentropy',
             weighted_metrics=['acc'])

model.summary()

# Callbacks
es_callback = EarlyStopping(monitor='val_weighted_acc', patience=es_patience)
# tb_callback = TensorBoard(batch_size=N)
mc_callback = ModelCheckpoint('logs/best_model.h5',
                              monitor='val_weighted_acc',
                              save_best_only=True,
                              save_weights_only=True)



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 2708)         0                                            
__________________________________________________________________________________________________
graph_attention_1 (GraphAttenti (None, 64)           91904       dropout_1[0][0]                  
                                                                 input_2[0][0]              

In [13]:
# Train model
validation_data = ([X,A],Y_val,idx_val)
model.fit([X,A],
         Y_train,
        sample_weight=idx_train,
          epochs=epochs,
          batch_size=N,
          validation_data=validation_data,
          shuffle=False,
          callbacks=[es_callback,mc_callback])
# callbacks=[es_callback,tb_callback,mc_callback]
# Test model
eval_results = model.evaluate([X,A],
                             Y_test,
                             sample_weight=idx_test,
                             batch_size=N,
                             verbose=0)

print('Done.\n'
      'Test loss: {}\n'
      'Test accuracy: {}'.format(*eval_results))

Train on 2708 samples, validate on 2708 samples
Epoch 1/10
2708/2708 [==============================] - 2s 749us/step - loss: 0.1139 - acc: 0.1643 - val_loss: 0.3694 - val_acc: 0.3260
Epoch 2/10
2708/2708 [==============================] - 2s 755us/step - loss: 0.1126 - acc: 0.2071 - val_loss: 0.3683 - val_acc: 0.3160
Epoch 3/10
2708/2708 [==============================] - 2s 746us/step - loss: 0.1118 - acc: 0.1786 - val_loss: 0.3673 - val_acc: 0.3160
Epoch 4/10
2708/2708 [==============================] - 2s 722us/step - loss: 0.1108 - acc: 0.1643 - val_loss: 0.3668 - val_acc: 0.3160
Epoch 5/10
2708/2708 [==============================] - 2s 744us/step - loss: 0.1101 - acc: 0.2071 - val_loss: 0.3664 - val_acc: 0.3180
Epoch 6/10
2708/2708 [==============================] - 2s 734us/step - loss: 0.1099 - acc: 0.2000 - val_loss: 0.3663 - val_acc: 0.3220
Epoch 7/10
2708/2708 [==============================] - 2s 790us/step - loss: 0.1096 - acc: 0.2143 - val_loss: 0.3663 - val_acc: 0.3620
